<a href="https://colab.research.google.com/github/DarkOxygen123/SBU-Hackathon/blob/main/Main_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib
import gc

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.utils import shuffle

In [ ]:
from tqdm import tqdm
import tensorflow as tf

from sklearn.metrics import average_precision_score, roc_auc_score, confusion_matrix

from scipy.special import softmax

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
!pip install pytorch-tabnet
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 774.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow_addons.activations import sparsemax

## Reading final Files

In [ ]:
df_x = pd.read_pickle("/content/drive/MyDrive/Dataset/X_processed")
df_y = pd.read_pickle("/content/drive/MyDrive/Dataset/Y_processed")

### Normalizing the column values

In [ ]:
df_x = (df_x - df_x.mean()) / df_x.std()

In [ ]:
class_0_percentage = 0.65
class_1_percentage = 0.35

In [ ]:
'''
# Shuffle and select samples for class 0
class_0_samples = shuffle(df_x[df_y == 0], random_state=42).iloc[:int(len(df_x) * class_0_percentage)]

# Shuffle and select samples for class 1
class_1_samples = shuffle(df_x[df_y == 1], random_state=42).iloc[:int(len(df_x) * class_1_percentage)]

# Combine the samples for both classes
X_train_balanced = pd.concat([class_0_samples, class_1_samples])
y_train_balanced = df_y.loc[X_train_balanced.index]

# Shuffle the combined dataset
X_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_train_balanced, random_state=42)

# Convert the balanced data to PyTorch tensors
X_train_balanced_tensor = torch.tensor(X_train_balanced.values, dtype=torch.float32)
y_train_balanced_tensor = torch.tensor(y_train_balanced.values, dtype=torch.int64)
'''

'\n# Shuffle and select samples for class 0\nclass_0_samples = shuffle(df_x[df_y == 0], random_state=42).iloc[:int(len(df_x) * class_0_percentage)]\n\n# Shuffle and select samples for class 1\nclass_1_samples = shuffle(df_x[df_y == 1], random_state=42).iloc[:int(len(df_x) * class_1_percentage)]\n\n# Combine the samples for both classes\nX_train_balanced = pd.concat([class_0_samples, class_1_samples])\ny_train_balanced = df_y.loc[X_train_balanced.index]\n\n# Shuffle the combined dataset\nX_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_train_balanced, random_state=42)\n\n# Convert the balanced data to PyTorch tensors\nX_train_balanced_tensor = torch.tensor(X_train_balanced.values, dtype=torch.float32)\ny_train_balanced_tensor = torch.tensor(y_train_balanced.values, dtype=torch.int64)\n'

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(377945, 371)
(377945,)
(118108, 371)
(118108,)
(94487, 371)
(94487,)


In [ ]:
X_train.head()

In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.int64)

## Training the Model

In [ ]:
X_train_tensor = X_train_tensor.numpy()
X_val_tensor = X_val_tensor.numpy()

In [ ]:
y_train_tensor = y_train_tensor.numpy()
y_val_tensor = y_val_tensor.numpy()

In [ ]:
del df_x
del df_y
del X_train
del X_val
del y_train
del y_val

In [ ]:
gc.collect()

5

In [ ]:
tabnet_model = TabNetClassifier()
tabnet_model.fit(
    X_train_tensor, y_train_tensor,
    eval_set=[(X_train_tensor, y_train_tensor), (X_val_tensor, y_val_tensor)],
    eval_name=['train', 'test'],
    eval_metric=['accuracy'],
    max_epochs=10
)

epoch 0  | loss: 0.14517 | train_accuracy: 0.96593 | test_accuracy: 0.96624 |  0:00:34s
epoch 1  | loss: 0.12475 | train_accuracy: 0.96914 | test_accuracy: 0.9692  |  0:01:09s
epoch 2  | loss: 0.12126 | train_accuracy: 0.96884 | test_accuracy: 0.96873 |  0:01:42s
epoch 3  | loss: 0.11924 | train_accuracy: 0.9675  | test_accuracy: 0.96776 |  0:02:15s
epoch 4  | loss: 0.11687 | train_accuracy: 0.97009 | test_accuracy: 0.97    |  0:02:50s
epoch 5  | loss: 0.1112  | train_accuracy: 0.97101 | test_accuracy: 0.97075 |  0:03:23s
epoch 6  | loss: 0.10924 | train_accuracy: 0.97172 | test_accuracy: 0.97123 |  0:03:56s
epoch 7  | loss: 0.1065  | train_accuracy: 0.97218 | test_accuracy: 0.97159 |  0:04:31s
epoch 8  | loss: 0.10408 | train_accuracy: 0.97305 | test_accuracy: 0.97241 |  0:05:04s
epoch 9  | loss: 0.10289 | train_accuracy: 0.97265 | test_accuracy: 0.9721  |  0:05:39s
Stop training because you reached max_epochs = 10 with best_epoch = 8 and best_test_accuracy = 0.97241


KeyboardInterrupt: 

In [ ]:
print(tabnet_model)

TabNetClassifier(n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=371, output_dim=2, device_name='auto', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[])


### Testing the model on datapoints which are unseen

In [ ]:
biased_class = 1  # Replace with the class you want to bias towards
bias_fraction = 0.1  # Adjust the fraction of biased samples

# Separate the test set into two parts: biased and unbiased
biased_mask = y_test == biased_class
unbiased_mask = ~biased_mask

# Shuffle the biased samples
X_test = shuffle(X_test[biased_mask], random_state=42)
y_test = shuffle(y_test[biased_mask], random_state=42)

In [ ]:
X_test, y_test = shuffle(X_test, y_test, random_state=31)

In [ ]:
X_test = torch.tensor(X_test.values, dtype=torch.int64)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
test_preds = tabnet_model.predict(X_test.numpy())
test_acc = accuracy_score(y_test, test_preds)
print(f"Test accuracy: {test_acc}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, test_preds)
print("Confusion Matrix:")
print(conf_matrix)

Test accuracy: 0.9721125657497857
Confusion Matrix:
[[90847   392]
 [ 2243  1005]]


In [ ]:
len(y_test)

In [ ]:
sum(y_test)

In [ ]:
sum(y_test)